<a href="https://colab.research.google.com/github/yimstar9/Engineer-Big-Data-Analysis_R/blob/main/%5BR%5DT2_1_TITANIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 생존여부 예측모델 만들기
### 학습용 데이터 (X_train, y_train)을 이용하여 생존 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 accuracy 평가지표에 따라 채점)

(가) 제공 데이터 목록
- y_train: 생존여부(학습용)
- X_trian, X_test : 승객 정보 (학습용 및 평가용)

(나) 데이터 형식 및 내용
- y_trian (712명 데이터)

**시험환경 세팅은 예시문제와 동일한 형태의 X_train, y_train, X_test 데이터를 만들기 위함임**

### 유의사항
- 성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, 피처엔지니어링, 분류알고리즘, 하이퍼파라미터 튜닝, 모형 앙상블 등이 수반되어야 한다.
- 수험번호.csv파일이 만들어지도록 코드를 제출한다.
- 제출한 모델의 성능은 accuracy로 평가함

csv 출력형태




In [ ]:
library(dplyr)
library(randomForest)
library(ModelMetrics)
df<-read.csv('../input/titanic/train.csv')
test<-read.csv('../input/titanic/test.csv')

In [ ]:
colSums(is.na(df)) #age컬럼에 결측값 있음
summary(df)
str(df)

PassengerId    Survived      Pclass        Name         Sex         Age 
          0           0           0           0           0         177 
      SibSp       Parch      Ticket        Fare       Cabin    Embarked 
          0           0           0           0           0           0

  PassengerId       Survived          Pclass          Name          
 Min.   :  1.0   Min.   :0.0000   Min.   :1.000   Length:891        
 1st Qu.:223.5   1st Qu.:0.0000   1st Qu.:2.000   Class :character  
 Median :446.0   Median :0.0000   Median :3.000   Mode  :character  
 Mean   :446.0   Mean   :0.3838   Mean   :2.309                     
 3rd Qu.:668.5   3rd Qu.:1.0000   3rd Qu.:3.000                     
 Max.   :891.0   Max.   :1.0000   Max.   :3.000                     
                                                                    
     Sex                 Age            SibSp           Parch       
 Length:891         Min.   : 0.42   Min.   :0.000   Min.   :0.0000  
 Class :character   1st Qu.:20.12   1st Qu.:0.000   1st Qu.:0.0000  
 Mode  :character   Median :28.00   Median :0.000   Median :0.0000  
                    Mean   :29.70   Mean   :0.523   Mean   :0.3816  
                    3rd Qu.:38.00   3rd Qu.:1.000   3rd Qu.:0.0000  
                    Max.   :80.00 

'data.frame':	891 obs. of  12 variables:
 $ PassengerId: int  1 2 3 4 5 6 7 8 9 10 ...
 $ Survived   : int  0 1 1 1 0 0 0 0 1 1 ...
 $ Pclass     : int  3 1 3 1 3 3 1 3 3 2 ...
 $ Name       : chr  "Braund, Mr. Owen Harris" "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" "Heikkinen, Miss. Laina" "Futrelle, Mrs. Jacques Heath (Lily May Peel)" ...
 $ Sex        : chr  "male" "female" "female" "female" ...
 $ Age        : num  22 38 26 35 35 NA 54 2 27 14 ...
 $ SibSp      : int  1 1 0 1 0 0 0 3 0 1 ...
 $ Parch      : int  0 0 0 0 0 0 0 1 2 0 ...
 $ Ticket     : chr  "A/5 21171" "PC 17599" "STON/O2. 3101282" "113803" ...
 $ Fare       : num  7.25 71.28 7.92 53.1 8.05 ...
 $ Cabin      : chr  "" "C85" "" "C123" ...
 $ Embarked   : chr  "S" "C" "S" "S" ...


## Start

In [ ]:
#전처리

df$Sex<-as.factor(df$Sex)
df$Embarked<-as.factor(df$Embarked)
df$Survived<-as.factor(df$Survived)
df$Pclass<-as.factor(df$Pclass)

test$Sex<-as.factor(test$Sex)
test$Embarked<-as.factor(test$Embarked)
test$Pclass<-as.factor(test$Pclass)

df$Age<-coalesce(df$Age,median(df$Age,na.rm=T))
test$Age<-coalesce(test$Age,median(test$Age,na.rm=T))
#age 결측값 중앙값으로대체
df<-df[,-c(4,9,10,11)]
str(test)
test<-test[,-c(3,8,9,10)]

idx<-sample((1:nrow(df)),nrow(df)*0.7)
x_train<-df[idx,]
x_test<-df[-idx,]



'data.frame':	418 obs. of  7 variables:
 $ PassengerId: int  892 893 894 895 896 897 898 899 900 901 ...
 $ Pclass     : Factor w/ 3 levels "1","2","3": 3 3 2 3 3 3 3 2 3 3 ...
 $ Sex        : Factor w/ 2 levels "female","male": 2 1 2 2 1 2 1 2 1 2 ...
 $ Age        : num  34.5 47 62 27 22 14 30 26 18 21 ...
 $ SibSp      : int  0 1 0 0 1 0 0 1 0 2 ...
 $ Parch      : int  0 0 0 0 1 0 0 1 0 0 ...
 $ Embarked   : Factor w/ 4 levels "","C","Q","S": 2 3 2 3 3 3 2 3 1 3 ...


In [ ]:
#모델링, 예측
m<-randomForest(Survived~.-(PassengerId),data=x_train, ntree=100)
p=predict(m,x_test)


In [ ]:
#평가 
#str(p)
#str(x_test)
result<-caret::confusionMatrix(x_test$Survived, p)$overall[1]
result


Accuracy 
0.8358209

In [ ]:
#최종 데이터 예측

levels(test$Embarked)<-levels(df$Embarked)
m1<-randomForest(Survived~.-(PassengerId),data=df, ntree=100)
p1=predict(m1,test)


ans<-cbind('Id'=test$Embarked,p1)
head(ans)

,Id,p1
1,2,1
2,3,1
3,2,1
4,3,1
5,3,1
6,3,2
